### API Yahoo Finance

In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
dict_Empresas = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Amazon" : "AMZN",
    "Alphabet": "GOOGL",


    # Algunas sugerencias:
    
    #"JPMorgan": "JPM",
    #"Johnson & Johnson": "JNJ",
    #"Walmart": "WMT"
    }

In [3]:
import os

if not os.path.exists('Datasets'):
    os.makedirs('Datasets')


for key, value in dict_Empresas.items():
    Dataset = yf.download(value, start=dt.datetime.now() - dt.timedelta(days=5*365), end=dt.date.today())
    Dataset = Dataset[["Close", "Volume"]]
    # Dataset["Ticker"] = value
    Dataset.to_csv(f"Datasets/{key}.csv")
    print(f"Archivo {key} cargado exitosamente.")


[*********************100%%**********************]  1 of 1 completed


Archivo Apple cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Microsoft cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Tesla cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Amazon cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed

Archivo Alphabet cargado exitosamente.


### SQL

In [4]:
import sqlite3
conn = sqlite3.connect('./Database/test.db')
conn.close()

In [5]:
Dataset

,Close,Volume
Date,,
2019-06-03,51.937000,96890000
2019-06-04,52.724499,60512000
2019-06-05,52.231998,46988000
2019-06-06,52.388000,29020000
2019-06-07,53.418499,43828000
...,...,...
2024-05-22,176.380005,17880000
2024-05-23,173.550003,21024900
2024-05-24,174.990005,16572500


In [6]:
import os

folder_path = './Datasets'
files = os.listdir(folder_path)

files

['Amazon.csv', 'Alphabet.csv', 'Apple.csv', 'Tesla.csv', 'Microsoft.csv']

In [7]:
dfs = []
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['Empresa'] = file.split('.')[0]
        dfs.append(df)


result = pd.concat(dfs, ignore_index=True)
result.head()

,Date,Close,Volume,Empresa
0,2019-06-03,84.634499,181974000,Amazon
1,2019-06-04,86.477997,113582000,Amazon
2,2019-06-05,86.925003,84796000,Amazon
3,2019-06-06,87.718002,73786000,Amazon
4,2019-06-07,90.201500,96164000,Amazon


In [8]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
conn.close()

Connected to SQLite


In [9]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='append')
conn.close()

Connected to SQLite


In [10]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='replace')

Connected to SQLite


6285

In [11]:
def write_sql_main(DB, df, table):
	"""Esta funcion inserta masivamente un df o una lista de tuplas en SQL."""
	try:
		conn = sqlite3.connect(DB)
		print("Connected to SQLite")
		df.to_sql(table, conn, schema=None, if_exists='replace', index=False, index_label=None, chunksize=None, dtype=None, method=None)
		result = 'ok'
	except sqlite3.Error as error:
		print("Failed to insert multiple records into sqlite table", error)
		result = 'failed'
	finally:
		if conn:
			conn.close()
			print("The SQLite connection is closed")
	return result

In [12]:
write_sql_main(DB=DB, df = result, table = 'trabajo_practico_yhf')

Connected to SQLite
The SQLite connection is closed


'ok'